# Code Structure:

**Load and Preprocess Data:** This section loads the data from a CSV file, encodes categorical columns, and scales the features.

**Split Data into Train and Test:** The dataset is split into training and test sets (80% training, 20% testing).

**Build and Train Random Forest Model:** A Random Forest classifier is trained on the training data using the default hyperparameters.

**Make Predictions and Evaluate the Model:** The model makes predictions on the test data, and evaluation metrics such as accuracy, precision, recall, and F1 score are calculated and displayed.

This Colab contain both XGBoost and Randomforest algorithm

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd

# Load the dataset
data = pd.read_csv('train_df.csv')

# Separate features and target
X = data.drop(columns=['target'])
y = data['target']

# Encode categorical columns
label_encoders = {}
categorical_cols = X.select_dtypes(include=['object']).columns
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Build and train the Random Forest model with default hyperparameters
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Evaluation Summary (Baseline Model):")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Evaluation Summary (Baseline Model):
Accuracy: 0.83
Precision: 0.82
Recall: 0.85
F1 Score: 0.83

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.81      0.83      1031
           1       0.82      0.85      0.83      1043

    accuracy                           0.83      2074
   macro avg       0.83      0.83      0.83      2074
weighted avg       0.83      0.83      0.83      2074



The evaluation results from your Random Forest model show that the model performs well, with the following key metrics:

**Evaluation Summary (Baseline Model):**

**Accuracy: 0.83** — This means the model correctly predicted 83% of the instances in the test set.

**Precision: 0.82** — This indicates the proportion of true positives out of all the instances predicted as positive.

**Recall: 0.85** — This is the proportion of actual positive instances correctly identified by the model.

**F1 Score: 0.83**  — This combines precision and recall into a single metric, where a value closer to 1 indicates a better model performance.

**further improve it by hyperparameter tuning (e.g., using RandomizedSearchCV or GridSearchCV)**

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import RandomizedSearchCV


# Base model class that is compatible with different classifiers
class BaseModel:
    def __init__(self):
        self.model = None
        self.scaler = None
        self.label_encoders = {}
        self.x_train, self.x_test, self.y_train, self.y_test = None, None, None, None
        self.data = None
        self.feature_columns = None

    def load(self, file_path):
        """Load dataset from a CSV file."""
        self.data = pd.read_csv(file_path)

    def preprocess(self, target_column, drop_columns):
        """Preprocess the dataset: encode, scale, and split the data."""
        x = self.data.drop(columns=drop_columns)
        y = self.data[target_column]

        # Encode categorical columns
        categorical_cols = x.select_dtypes(include=['object']).columns
        for col in categorical_cols:
            le = LabelEncoder()
            x[col] = le.fit_transform(x[col])
            self.label_encoders[col] = le

        # Scale numerical features
        self.scaler = StandardScaler()
        x_scaled = self.scaler.fit_transform(x)

        # Split into train and test sets
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)

        # Save feature columns for later use in prediction
        self.feature_columns = list(x.columns)

    def evaluate(self):
        """Evaluate the model on the test set and generate an evaluation summary."""
        y_pred = self.model.predict(self.x_test)
        accuracy = accuracy_score(self.y_test, y_pred)
        precision = precision_score(self.y_test, y_pred)
        recall = recall_score(self.y_test, y_pred)
        f1 = f1_score(self.y_test, y_pred)

        print("Evaluation Summary:")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print("\nClassification Report:")
        print(classification_report(self.y_test, y_pred))

    def predict(self, input_data):
        """Make predictions on preprocessed input data."""
        return self.model.predict(input_data)

    def predict_on_input(self, input_file, drop_columns):
        """Load and preprocess a new input file, then make predictions."""
        print("Loading and preprocessing prediction input file...")
        input_data = pd.read_csv(input_file)

        # Preprocess input data (drop columns that are not needed)
        input_data_processed = input_data.drop(columns=drop_columns)

        # Encode categorical columns using the saved label encoders
        for col, le in self.label_encoders.items():
            if col in input_data_processed.columns:
                input_data_processed[col] = le.transform(input_data_processed[col])

        # Ensure that the features used for prediction match the training data features
        input_data_processed = input_data_processed.reindex(columns=self.feature_columns, fill_value=0)

        # Scale input data using the same scaler
        input_data_scaled = self.scaler.transform(input_data_processed)

        # Predict using the trained model
        predictions = self.predict(input_data_scaled)
        input_data['success'] = predictions
        input_data['success'] = input_data['success'].map({1: 'Top', 0: 'Flop'})

        print("Predictions completed.")
        return input_data


# RandomForest Model Class
class RandomForestModel(BaseModel):
    def __init__(self):
        super().__init__()

    def train(self):
        """Train the Random Forest model."""
        self.model = RandomForestClassifier(random_state=42)
        self.model.fit(self.x_train, self.y_train)

    def tune_hyperparameters(self):
        """Perform hyperparameter tuning using RandomizedSearchCV."""
        param_dist = {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
        randomized_search = RandomizedSearchCV(self.model, param_distributions=param_dist, n_iter=10, cv=3, n_jobs=-1, verbose=2)
        randomized_search.fit(self.x_train, self.y_train)

        self.model = randomized_search.best_estimator_
        print(f"Best Hyperparameters: {randomized_search.best_params_}")


# Example Usage for RandomForestModel:
# Instantiate the Random Forest model
rf_model = RandomForestModel()

# Load and preprocess the dataset
rf_model.load('train_df.csv')  # Replace with your dataset path
rf_model.preprocess(target_column='target', drop_columns=['target'])

# Train the Random Forest model before evaluating it
rf_model.train()

# Evaluate the model before hyperparameter tuning
print("Evaluating the model before hyperparameter tuning:")
rf_model.evaluate()

# Optionally, perform hyperparameter tuning
rf_model.tune_hyperparameters()

# Evaluate the model after hyperparameter tuning
print("\nEvaluating the model after hyperparameter tuning:")
rf_model.evaluate()



Evaluating the model before hyperparameter tuning:
Evaluation Summary:
Accuracy: 0.83
Precision: 0.82
Recall: 0.85
F1 Score: 0.83

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.81      0.83      1031
           1       0.82      0.85      0.83      1043

    accuracy                           0.83      2074
   macro avg       0.83      0.83      0.83      2074
weighted avg       0.83      0.83      0.83      2074

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best Hyperparameters: {'n_estimators': 50, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 10}

Evaluating the model after hyperparameter tuning:
Evaluation Summary:
Accuracy: 0.84
Precision: 0.81
Recall: 0.89
F1 Score: 0.85

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.79      0.83      1031
           1       0.81      0.89      0.85      1043

    accuracy                     

**Evaluation Before Hyperparameter Tuning:**

**Accuracy: 0.83**
**Precision: 0.82**
**Recall: 0.85**
**F1 Score: 0.83**


**Key Insights:**

The model performs decently with a **high recall (0.85)**, meaning it correctly identifies most of the positive cases **(class 1)**.

Precision is also solid at 0.82, suggesting the model is reasonably good at not misclassifying negative cases as positive.

**The F1 score of 0.83 indicates a balanced performance between precision and recall.**

**Evaluation After Hyperparameter Tuning:**

**Accuracy: 0.84**
**Precision: 0.81**
**Recall: 0.89**
**F1 Score: 0.85**


**Key Insights:**

**After hyperparameter tuning**, the model's accuracy improved slightly from** 0.83 to 0.84**.

The recall for class 1 increased significantly from 0.85 to 0.89, meaning the model is now even better at identifying positive cases.

The precision for class 0 decreased slightly, from 0.84 to 0.81, but this is offset by the improved recall for class 1, leading to an overall better F1 score (0.85).

**The model is now slightly more sensitive to class 1 (positive cases), which could be beneficial if identifying positives is more important in your context.**

In [11]:
# Import required libraries
import pandas as pd
from xgboost import XGBClassifier, DMatrix, train
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from itertools import product


# Base Model Class
class BaseModel:
    def __init__(self):
        self.model = None
        self.scaler = None
        self.label_encoders = {}
        self.x_train, self.x_test, self.y_train, self.y_test = None, None, None, None
        self.data = None
        self.feature_columns = None

    def load(self, file_path):
        """Load dataset from a CSV file."""
        self.data = pd.read_csv(file_path)

    def preprocess(self, target_column, drop_columns):
        """Preprocess the dataset: encode, scale, and split the data."""
        x = self.data.drop(columns=drop_columns)
        y = self.data[target_column]

        # Encode categorical columns
        categorical_cols = x.select_dtypes(include=['object']).columns
        for col in categorical_cols:
            le = LabelEncoder()
            x[col] = le.fit_transform(x[col])
            self.label_encoders[col] = le

        # Scale numerical features
        self.scaler = StandardScaler()
        x_scaled = self.scaler.fit_transform(x)

        # Split into train and test sets
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(
            x_scaled, y, test_size=0.2, random_state=42
        )

        # Save feature columns for later use in prediction
        self.feature_columns = list(x.columns)

    def evaluate(self):
        """Evaluate the model on the test set and generate an evaluation summary."""
        y_pred = self.model.predict(self.x_test)
        accuracy = accuracy_score(self.y_test, y_pred)
        precision = precision_score(self.y_test, y_pred)
        recall = recall_score(self.y_test, y_pred)
        f1 = f1_score(self.y_test, y_pred)

        print("Evaluation Summary:")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print("\nClassification Report:")
        print(classification_report(self.y_test, y_pred))


# XGBoost Model Class
class XGBoostModel(BaseModel):
    def __init__(self):
        super().__init__()

    def train(self):
        """Train the XGBoost model."""
        self.model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
        self.model.fit(self.x_train, self.y_train)

    def tune_hyperparameters(self):
        """Perform manual hyperparameter tuning using XGBoost with early stopping."""
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 6, 10],
            'learning_rate': [0.01, 0.1, 0.2],
            'subsample': [0.6, 0.8, 1.0],
            'colsample_bytree': [0.6, 0.8, 1.0]
        }

        # Convert the training data to DMatrix format
        dtrain = DMatrix(self.x_train, label=self.y_train)
        dtest = DMatrix(self.x_test, label=self.y_test)

        best_score = 0
        best_params = None

        # Generate all combinations of parameters
        param_combinations = [dict(zip(param_grid.keys(), values)) for values in product(*param_grid.values())]

        for params in param_combinations:
            print(f"Testing parameters: {params}")
            model_params = {
                'objective': 'binary:logistic',
                'eval_metric': 'logloss',
                'random_state': 42,
                **params
            }

            # Train with early stopping
            evals = [(dtrain, 'train'), (dtest, 'test')]
            model = train(
                model_params,
                dtrain,
                num_boost_round=1000,
                early_stopping_rounds=10,
                evals=evals,
                verbose_eval=False
            )

            score = model.eval(dtest)
            accuracy = 1 - float(score.split(':')[1])  # Convert logloss to accuracy

            print(f"Validation Accuracy: {accuracy:.4f}")

            if accuracy > best_score:
                best_score = accuracy
                best_params = params

        print(f"Best Hyperparameters: {best_params}")
        print(f"Best Validation Accuracy: {best_score:.4f}")

        # Set the best model
        self.model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss', **best_params)
        self.model.fit(self.x_train, self.y_train)


# Main Script
if __name__ == "__main__":
    # Load and preprocess data
    xgb_model = XGBoostModel()
    file_path = 'train_df.csv'  # Replace with your dataset path
    target_column = 'target'  # Replace with your target column name
    drop_columns = ['target']  # Columns to drop, including the target column

    # Load and preprocess
    xgb_model.load(file_path)
    xgb_model.preprocess(target_column=target_column, drop_columns=drop_columns)

    # Evaluate before tuning
    print("\nEvaluating XGBoost Model Before Hyperparameter Tuning:")
    xgb_model.train()
    xgb_model.evaluate()

    # Hyperparameter tuning
    print("\nPerforming Hyperparameter Tuning for XGBoost Model...")
    xgb_model.tune_hyperparameters()

    # Evaluate after tuning
    print("\nEvaluating XGBoost Model After Hyperparameter Tuning:")
    xgb_model.evaluate()



Evaluating XGBoost Model Before Hyperparameter Tuning:


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [17:26:24] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Evaluation Summary:
Accuracy: 0.84
Precision: 0.83
Recall: 0.87
F1 Score: 0.85

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.82      0.84      1031
           1       0.83      0.87      0.85      1043

    accuracy                           0.84      2074
   macro avg       0.84      0.84      0.84      2074
weighted avg       0.84      0.84      0.84      2074


Performing Hyperparameter Tuning for XGBoost Model...
Testing parameters: {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.01, 'subsample': 0.6, 'colsample_bytree': 0.6}


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [17:26:25] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


Validation Accuracy: 0.5606
Testing parameters: {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.01, 'subsample': 0.6, 'colsample_bytree': 0.8}


...

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [17:30:07] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


Validation Accuracy: 0.6096
Testing parameters: {'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.2, 'subsample': 0.8, 'colsample_bytree': 1.0}
Validation Accuracy: 0.6037
Testing parameters: {'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.2, 'subsample': 1.0, 'colsample_bytree': 0.6}
Validation Accuracy: 0.5965
Testing parameters: {'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.2, 'subsample': 1.0, 'colsample_bytree': 0.8}


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [17:30:08] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


Validation Accuracy: 0.6121
Testing parameters: {'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.2, 'subsample': 1.0, 'colsample_bytree': 1.0}
Validation Accuracy: 0.6083
Best Hyperparameters: {'n_estimators': 50, 'max_depth': 6, 'learning_rate': 0.1, 'subsample': 1.0, 'colsample_bytree': 0.8}
Best Validation Accuracy: 0.6154

Evaluating XGBoost Model After Hyperparameter Tuning:
Evaluation Summary:
Accuracy: 0.84
Precision: 0.82
Recall: 0.88
F1 Score: 0.85

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.81      0.84      1031
           1       0.82      0.88      0.85      1043

    accuracy                           0.84      2074
   macro avg       0.84      0.84      0.84      2074
weighted avg       0.84      0.84      0.84      2074



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [17:30:09] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [17:30:09] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:


# Load and preprocess the training data for both models
rf_pipeline = RandomForestModel()
rf_pipeline.load('train_df.csv')
rf_pipeline.preprocess(target_column='target', drop_columns=['target'])
rf_pipeline.train()

xgb_pipeline = XGBoostModel()
xgb_pipeline.load('train_df.csv')
xgb_pipeline.preprocess(target_column='target', drop_columns=['target'])
xgb_pipeline.train()

# Load prediction input data
prediction_input = pd.read_csv('prediction_input.csv')

# Preprocess and predict with Random Forest
rf_predictions = rf_pipeline.predict_on_input('prediction_input.csv', drop_columns=['item_no'])
# Check if Random Forest predictions were added correctly
print("Random Forest Predictions:")
print(rf_predictions.head())

# Preprocess and predict with XGBoost
xgb_predictions = xgb_pipeline.predict_on_input('prediction_input.csv', drop_columns=['item_no'])
# Check if XGBoost predictions were added correctly
print("XGBoost Predictions:")
print(xgb_predictions.head())

# Ensure the predictions are added
rf_predictions['success_rf'] = rf_predictions['success']
xgb_predictions['success_xgb'] = xgb_predictions['success']

# # Check if both prediction columns exist before final output
# print("Final DataFrame with both predictions:")
# print(rf_predictions[['item_no', 'category', 'main_promotion', 'color', 'stars', 'success_rf']].head())
# print(xgb_predictions[['item_no', 'category', 'main_promotion', 'color', 'stars', 'success_xgb']].head())

# Merge predictions from both models into one DataFrame
final_predictions = pd.merge(rf_predictions, xgb_predictions[['item_no', 'success_xgb']], on='item_no', how='left')

# Final output
print("Prediction Output:")
print(final_predictions[['item_no', 'category', 'main_promotion', 'color', 'stars', 'success_rf', 'success_xgb']])

Loading and preprocessing prediction input file...
Predictions completed.
Random Forest Predictions:
   item_no    category    main_promotion   color  stars success
0   405901  Sweatshirt           Catalog    Blue    3.1     Top
1   644275  Polo-Shirt  Frontpage_Header  Yellow    2.6    Flop
2   533070       Tunic           Catalog   Green    2.7    Flop
3   829436  Polo-Shirt           Catalog  Yellow    2.6    Flop
4   801722       Tunic           Catalog  Yellow    4.9    Flop
Loading and preprocessing prediction input file...
Predictions completed.
XGBoost Predictions:
   item_no    category    main_promotion   color  stars success
0   405901  Sweatshirt           Catalog    Blue    3.1     Top
1   644275  Polo-Shirt  Frontpage_Header  Yellow    2.6    Flop
2   533070       Tunic           Catalog   Green    2.7    Flop
3   829436  Polo-Shirt           Catalog  Yellow    2.6    Flop
4   801722       Tunic           Catalog  Yellow    4.9     Top
Prediction Output:
      item_no    

In [ ]:
final_predictions.head(10)

,item_no,category,main_promotion,color,stars,success,success_rf,success_xgb
0,405901,Sweatshirt,Catalog,Blue,3.1,Top,Top,Top
1,644275,Polo-Shirt,Frontpage_Header,Yellow,2.6,Flop,Flop,Flop
2,533070,Tunic,Catalog,Green,2.7,Flop,Flop,Flop
3,829436,Polo-Shirt,Catalog,Yellow,2.6,Flop,Flop,Flop
4,801722,Tunic,Catalog,Yellow,4.9,Flop,Flop,Top
5,866263,T-Shirt,Category_Highlight,Black,2.6,Flop,Flop,Flop
6,502221,Sweatshirt,Catalog,Red,1.6,Flop,Flop,Flop
7,545865,Tunic,Category_Highlight,Green,3.5,Top,Top,Top
8,440112,Sweatshirt,Display_Ad_Campaign,Blue,3.7,Top,Top,Top
9,930925,Tunic,Catalog,Green,2.0,Flop,Flop,Flop


This is the prediction with prediction input data for 2000 data points

# **Conclusion**

**Before tuning:**
The model had a solid baseline performance with good precision and recall, but there was still room to improve recall for class 1.

**After tuning:**
The hyperparameter tuning improved recall for class 1, making the model more sensitive to identifying positive cases, which led to a small increase in accuracy and a higher F1 score.